# Hypothesis

This notebook serves as a demo of the basic features and functionality of the Python tool **Hypothesis**.

## What is Hypothesis

Hypothesis is a tool that allows us to write and generate tests that generate and allow for fuzzing over a large variety of data without actually manually making all the tests. The key features mentioned are **property-based-testing** using `@given`, **strategies** for primitive and more complex data types, **shrinking** to find the simplest possible counter-example, and **stateful testing** to test with defined rules and checking invariants.

## 1. Basic Usage: `@given` and Strategies

The core of Hypothesis is the `@given` decorator, which takes **strategies** that define how to generate data. Let's test a simple `add` function.

In [1]:
from hypothesis import given, strategies as st

def add(a, b):
    """A simple function to add two numbers."""
    return a + b

# This is a property-based test!
# It states that for any two integers a and b, add(a, b) should equal a + b.

@given(a=st.integers(), b=st.integers())
def test_addition_property(a, b):
    print(f"Testing with a={a}, b={b}") 
    assert add(a, b) == a + b

test_addition_property()

Testing with a=0, b=0
Testing with a=-1880641520, b=0
Testing with a=-1880641520, b=-24144
Testing with a=-1145854508, b=0
Testing with a=-1145854508, b=-4552
Testing with a=-22908, b=0
Testing with a=-22908, b=-13637
Testing with a=21240, b=0
Testing with a=21240, b=-89
Testing with a=113, b=0
Testing with a=113, b=-617485230
Testing with a=113, b=113
Testing with a=-383, b=-75
Testing with a=-75, b=-75
Testing with a=-24, b=-123
Testing with a=-123, b=-123
Testing with a=-6964901394039317262, b=30714
Testing with a=30714, b=30714
Testing with a=8, b=-10154
Testing with a=-10154, b=-10154
Testing with a=-22913, b=1791852889180407179
Testing with a=1791852889180407179, b=1791852889180407179
Testing with a=-32130, b=6814578567113010312
Testing with a=-32130, b=-32130
Testing with a=-4001800936546175225, b=-13313
Testing with a=-4001800936546175225, b=-4001800936546175225
Testing with a=-4532316777546885352, b=16221
Testing with a=-4532316777546885352, b=-4532316777546885352
Testing with

Notice how Hypothesis tries a wide range of integers: positives, negatives, and zero. If the assertion had failed, Hypothesis would have reported it.

## 2. Shrinking, Counter-Examples, and Replaying

Hypothesis's superpower is finding the *simplest possible* failing example. This is called **shrinking**. Let's introduce a bug into a sorting function.

In [2]:
import shutil
try:
    shutil.rmtree('.hypothesis')
except:
    print('no cache')


no cache


In [ ]:
from hypothesis import find, settings, note
from typing import List

def buggy_sort(numbers: List[int]) -> List[int]:
    if 0 in numbers and 1 in numbers:
        return [999] #oh no our sort is wrong
    return sorted(numbers)

# defined strategy
list_strategy = st.lists(
    st.integers(min_value=-5, max_value=5),
    min_size=1,
    max_size=10
)

# ignore
seen_inputs = []

#ignore
def check_sort(numbers: List[int]):
    seen_inputs.append(numbers[:])  
    note(f"Checking example: {numbers}")
    assert buggy_sort(numbers) == sorted(numbers)

# --- Settings ---
@settings(verbosity=2, max_examples=1000, deadline=None)
def run_find():
    print("Searching for example")
    failing_example = find(list_strategy, check_sort)
    print(f" {failing_example}")

run_find()

In [ ]:
import matplotlib.pyplot as plt

sizes = [len(x) for x in seen_inputs]
plt.figure(figsize=(10, 5))
plt.plot(sizes, marker='o', linestyle='-', color='blue', label='Input size')
plt.xlabel('Test number')
plt.ylabel('Input size (length of list)')
plt.title(' Shrinking Behavior in Hypothesis')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

You might note it shrank it down to the simplest possible case that still causes the error.

#### Replaying Failures

When a test fails, Hypothesis saves the failing example. You can use the `@example` decorator to add it as a permanent regression test case, ensuring the bug never comes back.

In [ ]:
from hypothesis import given, example


@given(st.lists(st.integers()))
@example([0, 1])  
def test_buggy_sort(numbers):
    try:
        assert buggy_sort(numbers) == sorted(numbers)
    except AssertionError:
        print(f"Test failed for input: {numbers}")

test_buggy_sort()

print("Test with @example ran.")


### Note
Hypothesis database may be invalidated when using different Hypothesis versions or when changing the source code of test functions. Instead of relying on the Hypothesis to replay the last failure from the database, it may be preferred to use `@example` in these cases.

## Filtering and Transformations (`.filter()`, `assume`, `.map()`)

Hypothesis provides several ways to narrow down the set of possible examples to generate.

In [ ]:
from hypothesis import assume

count_examples = {"filter": 0, "assume": 0, "early return": 0}
count_pass = {"filter": 0, "assume": 0, "early return": 0}

@settings(max_examples=50)
@given(st.integers().filter(lambda x: x % 2 == 0), st.integers().filter(lambda x: x % 2 == 0))
def test_filter(a, b):
    count_examples["filter"] += 1

    assert (a+b) % 2 == 0

    count_pass["filter"] += 1

@settings(max_examples=50)
@given(st.integers(), st.integers())
def test_assume(a, b):
    count_examples["assume"] += 1

    assume(a % 2 == 0 and b % 2 == 0)
    assert (a+b) % 2 == 0

    count_pass["assume"] += 1

@settings(max_examples=50)
@given(st.integers(), st.integers())
def test_early_return(a, b):
    count_examples["early return"] += 1

    if a % 2 != 0 or b % 2 != 0:
        return
    assert (a+b) % 2 == 0

    count_pass["early return"] += 1

test_filter()
test_assume()
test_early_return()

print(f"{'method':15s} | {'examples created':20s} | {'success rate':>11s}")
print("="*60)
for method in "filter", "assume", "early return":
    print(f"{method:15s} | {count_examples[method]:20d} | {count_pass[method]:10d}/50")

In [ ]:
import time

@settings(max_examples=500)
@given(st.integers().map(lambda x: x*2), st.integers().map(lambda x: x*2))
def test_mapping(a, b):
    assert (a+b) % 2 == 0

@settings(max_examples=500)
@given(st.integers().filter(lambda x: x % 2 == 0), st.integers().filter(lambda x: x % 2 == 0))
def test_filtering(a, b):
    assert (a+b) % 2 == 0

for method, fn in ("mapping", test_mapping), ("filtering", test_filtering):
    start_time = time.perf_counter()
    fn()
    end_time = time.perf_counter()
    elapsed = end_time - start_time
    print(f"{method:8s} | time: {elapsed:8.4f}s")

### Takeaways
1. A successful example is one where an assert does not fail which is why early return generates exactly 50 examples that are not considered failures.
2. `.assume()` will not make progress towards max examples if assumption fails, but will still invoke the function (think about side effects if assumption is in middle of body)
3. Should prefer `.filter()` to `.assume()` and avoid early return when using Hypothesis.
4. Consider using mapping over filtering for potential perfomance gains.

Hypothesis also claims that simple `.filter()` can be sometimes be optimized beyond trivial rejection sampling unlike `.assume()`

## 4. Custom and Composite Strategies

If you need to generate objects, you can use the `@composite` decorator, which lets you build new strategies from existing ones.

In [11]:
from hypothesis.strategies import composite
from dataclasses import dataclass

@dataclass
class LotteryTicket:
    participant_id: int
    ticket_hash: str
    
#draw() draws a value from another strategy.

@composite
def ticket_strategy(draw):
    """A strategy to generate Ticket objects."""
    participant_id = draw(st.integers(min_value=0000, max_value=9999))
    ticket_hash = draw(st.text(alphabet="0123456789abcdef", min_size=3, max_size=8))
    return LotteryTicket(participant_id=participant_id, ticket_hash=ticket_hash)

@given(ticket=ticket_strategy())
def test_ticket_creation(ticket):
    print(f"Generated Ticket: {ticket}")
    assert isinstance(ticket, LotteryTicket)
    assert ticket.participant_id >= 0000
    assert ticket.participant_id <= 9999
    assert len(ticket.ticket_hash) >= 3
    assert len(ticket.ticket_hash) <= 8

test_ticket_creation()

Generated Ticket: LotteryTicket(participant_id=0, ticket_hash='000')
Generated Ticket: LotteryTicket(participant_id=1738, ticket_hash='000')
Generated Ticket: LotteryTicket(participant_id=1738, ticket_hash='adac34')
Generated Ticket: LotteryTicket(participant_id=1273, ticket_hash='000')
Generated Ticket: LotteryTicket(participant_id=1273, ticket_hash='b4abe713')
Generated Ticket: LotteryTicket(participant_id=7819, ticket_hash='000')
Generated Ticket: LotteryTicket(participant_id=7819, ticket_hash='613')
Generated Ticket: LotteryTicket(participant_id=9999, ticket_hash='000')
Generated Ticket: LotteryTicket(participant_id=9999, ticket_hash='e0012')
Generated Ticket: LotteryTicket(participant_id=991, ticket_hash='000')
Generated Ticket: LotteryTicket(participant_id=991, ticket_hash='6db9d50')
Generated Ticket: LotteryTicket(participant_id=4389, ticket_hash='2ee47a8')
Generated Ticket: LotteryTicket(participant_id=7815, ticket_hash='c8a04b52')
Generated Ticket: LotteryTicket(participant_id

We can also generate a value that depends on a value from another strategy. 

In [24]:
@composite
def ordered_ticket_pair(draw):
    #first ticket
    t1_id = draw(st.integers(min_value=0, max_value=9998)) #1 less in case t2_id is 9999
    t1_hash = draw(st.text(alphabet="0123456789abcdef", min_size=3, max_size=8))
    ticket1 = LotteryTicket(participant_id=t1_id, ticket_hash=t1_hash)
    
    #second ticket, where its participant_id > first ticket's participant_id
    t2_id = draw(st.integers(min_value=t1_id+1, max_value=9999))
    t2_hash = draw(st.text(alphabet="0123456789abcdef", min_size=3, max_size=8))
    ticket2 = LotteryTicket(participant_id=t2_id, ticket_hash=t2_hash)
    
    return (ticket1, ticket2)

@given(ordered_ticket_pair())
def test_ordered_tickets(pair):
    ticket1, ticket2 = pair
    print(ticket1, ticket2)
    #the second participant_id is > the first
    assert ticket1.participant_id < ticket2.participant_id

    for t in pair:
        assert 3 <= len(t.ticket_hash) <= 8

test_ordered_tickets()

LotteryTicket(participant_id=0, ticket_hash='000') LotteryTicket(participant_id=4619, ticket_hash='c0fa')
LotteryTicket(participant_id=0, ticket_hash='9c5dc6d0') LotteryTicket(participant_id=372, ticket_hash='47df8cb')
LotteryTicket(participant_id=0, ticket_hash='000') LotteryTicket(participant_id=1, ticket_hash='000')
LotteryTicket(participant_id=3648, ticket_hash='000') LotteryTicket(participant_id=3649, ticket_hash='000')
LotteryTicket(participant_id=3648, ticket_hash='84f5') LotteryTicket(participant_id=9624, ticket_hash='43f')
LotteryTicket(participant_id=2639, ticket_hash='000') LotteryTicket(participant_id=2640, ticket_hash='000')
LotteryTicket(participant_id=2639, ticket_hash='e7f3') LotteryTicket(participant_id=6513, ticket_hash='0de04202')
LotteryTicket(participant_id=8278, ticket_hash='000') LotteryTicket(participant_id=8279, ticket_hash='000')
LotteryTicket(participant_id=8278, ticket_hash='bf0f3') LotteryTicket(participant_id=8916, ticket_hash='40edb')
LotteryTicket(partic

`@data()` is an alternative to `@composite()` if you don't want to finish generating the entire input before running the test. Some cases of this are if 
1. You don't want to generate all the input until some initial assertions have passed.
2. You have some complicated control flow which makes it necessary to generate something in the middle of the test.

`@data()` lets you mix test code and generation code. You can dynamically call data.draw(strategy) any number of times inside the test. Each draw is independent except when you pass dependent parameters, and you don't need a separate `@composite` strategy if all your drawing logic fits in one test.

In [ ]:
from hypothesis.strategies import data

@given(data())
def test_ordered_tickets(data):
    t1_id = data.draw(st.integers(min_value=0, max_value=9998)) #1 less in case t2_id is 9999
    t1_hash = data.draw(st.text(alphabet="0123456789abcdef", min_size=3, max_size=8))
    ticket1 = LotteryTicket(participant_id=t1_id, ticket_hash=t1_hash)

    t2_id = data.draw(st.integers(min_value=t1_id+1, max_value=9999))
    t2_hash = data.draw(st.text(alphabet="0123456789abcdef", min_size=3, max_size=8))
    ticket2 = LotteryTicket(participant_id=t2_id, ticket_hash=t2_hash)

    print(ticket1, ticket2)
    assert ticket1.participant_id < ticket2.participant_id
    for t in (ticket1, ticket2):
        assert 3 <= len(t.ticket_hash) <= 8

test_ordered_tickets()








LotteryTicket(participant_id=0, ticket_hash='000') LotteryTicket(participant_id=1, ticket_hash='000')
LotteryTicket(participant_id=4665, ticket_hash='000') LotteryTicket(participant_id=4666, ticket_hash='000')
LotteryTicket(participant_id=4665, ticket_hash='308c') LotteryTicket(participant_id=6076, ticket_hash='266')
LotteryTicket(participant_id=1, ticket_hash='000') LotteryTicket(participant_id=2, ticket_hash='000')
LotteryTicket(participant_id=1, ticket_hash='0e6ca2a') LotteryTicket(participant_id=7422, ticket_hash='eee31c5')
LotteryTicket(participant_id=5950, ticket_hash='000') LotteryTicket(participant_id=5951, ticket_hash='000')
LotteryTicket(participant_id=5950, ticket_hash='1ac') LotteryTicket(participant_id=9842, ticket_hash='d0d')
LotteryTicket(participant_id=5940, ticket_hash='000') LotteryTicket(participant_id=5941, ticket_hash='000')
LotteryTicket(participant_id=5940, ticket_hash='48173a3a') LotteryTicket(participant_id=8585, ticket_hash='dd07')
LotteryTicket(participant_id

Using `@composite`, we can also add parameters to functions, including keyword-only arguments.

In [21]:
from hypothesis.strategies import composite

@composite
def lottery_tickets(draw, count=1):
    """Generate a `count`-sized list of LotteryTicket objects."""
    participant_ids = draw(st.lists(st.integers(min_value=0000, max_value=9999), min_size=count, max_size=count, unique=True))
    ticket_hashes = draw(st.lists(st.text(alphabet="0123456789abcdef", min_size=8, max_size=8), min_size=count, max_size=count))

    tickets = []
    for p_id, t_hash in zip(participant_ids, ticket_hashes):
        tickets.append(LotteryTicket(p_id, t_hash))
    return tickets

@given(lottery_tickets(3))
def test_lottery_ticket_generation(tickets):
    assert len(tickets) == 3
    print("-----")
    for t in tickets:
        print(f"Generated Ticket: {t}")
        assert isinstance(t.participant_id, int)
        assert isinstance(t.ticket_hash, str)
        assert all(c in "0123456789abcdef" for c in t.ticket_hash)

@given(lottery_tickets(count=5))
def test_lottery_ticket_generation_keyworded(tickets):
    assert len(tickets) == 5
    print("-----")
    for t in tickets:
        print(f"Generated Ticket (keyworded): {t}")
        assert isinstance(t.participant_id, int)
        assert isinstance(t.ticket_hash, str)
        assert all(c in "0123456789abcdef" for c in t.ticket_hash)

test_lottery_ticket_generation()
test_lottery_ticket_generation_keyworded()

-----
Generated Ticket: LotteryTicket(participant_id=271, ticket_hash='cfbe4c1b')
Generated Ticket: LotteryTicket(participant_id=6817, ticket_hash='c72d1223')
Generated Ticket: LotteryTicket(participant_id=2667, ticket_hash='7cff9c88')
-----
Generated Ticket: LotteryTicket(participant_id=2802, ticket_hash='e4434cda')
Generated Ticket: LotteryTicket(participant_id=9998, ticket_hash='4c07b354')
Generated Ticket: LotteryTicket(participant_id=6433, ticket_hash='dd9038b3')
-----
Generated Ticket: LotteryTicket(participant_id=2660, ticket_hash='1a35756b')
Generated Ticket: LotteryTicket(participant_id=5409, ticket_hash='e9f4d5e5')
Generated Ticket: LotteryTicket(participant_id=9566, ticket_hash='aabc1e37')
-----
Generated Ticket: LotteryTicket(participant_id=838, ticket_hash='bc121cfe')
Generated Ticket: LotteryTicket(participant_id=6496, ticket_hash='c4aa0984')
Generated Ticket: LotteryTicket(participant_id=8238, ticket_hash='f65b242e')
-----
Generated Ticket: LotteryTicket(participant_id=6

## 5. Stateful Testing

Hypothesis can even test stateful systems by generating sequences of actions and checking that invariants (rules that should always be true) hold.

Let's test a simple `SimpleStack` class.

In [ ]:
from hypothesis.stateful import RuleBasedStateMachine, rule, precondition

class SimpleStack:
    """A basic stack implementation."""
    def __init__(self):
        self._items = []
    
    def push(self, item):
        self._items.append(item)
        
    def pop(self):
        if not self._items:
            raise IndexError("pop from empty list")
        return self._items.pop()
    
    def is_empty(self):
        return not self._items
    
    @property
    def size(self):
        return len(self._items)

class StackStateMachine(RuleBasedStateMachine):
    """Defines the rules for testing our stack."""
    def __init__(self):
        super().__init__()
        self.stack = SimpleStack()
        self.model = [] # A simple list to model the stack's behavior
        
    @rule(item=st.integers())
    def push_item(self, item):
        """Rule for pushing an item."""
        self.stack.push(item)
        self.model.append(item)
        print(f"Pushed {item}")
        
    @rule()
    @precondition(lambda self: not self.stack.is_empty()) # Only run pop if not empty
    def pop_item(self):
        """Rule for popping an item."""
        popped_stack = self.stack.pop()
        popped_model = self.model.pop()
        print(f"Popped {popped_stack}")
        assert popped_stack == popped_model
    
    @rule()
    def check_invariants(self):
        """This rule checks properties that should always be true."""
        print("Checking invariants...")
        assert self.stack.size == len(self.model)
        assert self.stack.is_empty() == (not self.model)

# To run this, Hypothesis would execute a sequence of the rules defined above.
TestStack = StackStateMachine.TestCase
TestStack.runTest = lambda self: None # Suppress unittest's default runner

test_case = TestStack()
test_case.execute_step = test_case.execute_step
print("Running stateful test...")
# Manually run a few steps for demonstration
try:
    for i in range(10): # Run 10 random steps
        test_case.execute_step(test_case.steps.pop(0))
except (IndexError, AttributeError):
    pass # Stop if we run out of generated steps
print("\nStateful test finished.")